## Import data

Dataset was loaded from Kaggle: https://www.kaggle.com/datasets/nathanlauga/nba-games/data

### Games information

In [9]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.metrics import classification_report
from tqdm import tqdm
import numpy as np

df_games = pd.read_csv('data/games.csv').drop(["TEAM_ID_home", "TEAM_ID_away"], axis=1)
df_games = df_games.loc[df_games["SEASON"] >= 2004]

min_max_scaler = MinMaxScaler()
cols_to_norm = ["HOME_TEAM_ID", "VISITOR_TEAM_ID"]
df_games.loc[:, cols_to_norm] = min_max_scaler.fit_transform(df_games.loc[:, cols_to_norm])
# df_games.loc[:, ["HOME_TEAM_ID", "VISITOR_TEAM_ID"]] %= 100
df_games

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2022-12-22,22200477,Final,0.103448,0.758621,2022,126.0,0.484,0.926,0.382,25.0,46.0,117.0,0.478,0.815,0.321,23.0,44.0,1
1,2022-12-22,22200478,Final,0.862069,0.931034,2022,120.0,0.488,0.952,0.457,16.0,40.0,112.0,0.561,0.765,0.333,20.0,37.0,1
2,2022-12-21,22200466,Final,0.068966,0.413793,2022,114.0,0.482,0.786,0.313,22.0,37.0,106.0,0.470,0.682,0.433,20.0,46.0,1
3,2022-12-21,22200467,Final,0.620690,0.965517,2022,113.0,0.441,0.909,0.297,27.0,49.0,93.0,0.392,0.735,0.261,15.0,46.0,1
4,2022-12-21,22200468,Final,0.000000,0.137931,2022,108.0,0.429,1.000,0.378,22.0,47.0,110.0,0.500,0.773,0.292,20.0,47.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26646,2014-10-06,11400007,Final,0.000000,0.103448,2014,93.0,0.419,0.821,0.421,24.0,50.0,87.0,0.366,0.643,0.375,17.0,43.0,1
26647,2014-10-06,11400004,Final,0.137931,0.931034,2014,81.0,0.338,0.719,0.381,18.0,40.0,85.0,0.411,0.636,0.267,17.0,47.0,0
26648,2014-10-06,11400005,Final,0.344828,0.206897,2014,98.0,0.448,0.682,0.500,29.0,45.0,95.0,0.387,0.659,0.500,19.0,43.0,1
26649,2014-10-05,11400002,Final,0.827586,0.724138,2014,99.0,0.440,0.771,0.333,21.0,30.0,94.0,0.469,0.725,0.385,18.0,45.0,1


### Teams information

In [2]:
df_teams = pd.read_csv('data/teams.csv')
df_teams.loc[:, ["TEAM_ID"]] = min_max_scaler.fit_transform(df_teams.loc[:, ["TEAM_ID"]])
df_teams.head()

,LEAGUE_ID,TEAM_ID,MIN_YEAR,MAX_YEAR,ABBREVIATION,NICKNAME,YEARFOUNDED,CITY,ARENA,ARENACAPACITY,OWNER,GENERALMANAGER,HEADCOACH,DLEAGUEAFFILIATION
0,0,0.000000,1949,2019,ATL,Hawks,1949,Atlanta,State Farm Arena,18729.0,Tony Ressler,Travis Schlenk,Lloyd Pierce,Erie Bayhawks
1,0,0.034483,1946,2019,BOS,Celtics,1946,Boston,TD Garden,18624.0,Wyc Grousbeck,Danny Ainge,Brad Stevens,Maine Red Claws
2,0,0.103448,2002,2019,NOP,Pelicans,2002,New Orleans,Smoothie King Center,NaN,Tom Benson,Trajan Langdon,Alvin Gentry,No Affiliate
3,0,0.137931,1966,2019,CHI,Bulls,1966,Chicago,United Center,21711.0,Jerry Reinsdorf,Gar Forman,Jim Boylen,Windy City Bulls
4,0,0.172414,1980,2019,DAL,Mavericks,1980,Dallas,American Airlines Center,19200.0,Mark Cuban,Donnie Nelson,Rick Carlisle,Texas Legends


### Seasons dates

In [3]:
season_schedule = pd.read_csv("data/season_schedule.csv")
"""
SEASON: start year of the season, i.e. season 2004 is season 2004-2005
START: beginning date of regular mathces
END: beginning date of play-off
"""
season_schedule.head()

,SEASON,START,END
0,2004,2004-11-02,2005-04-23
1,2005,2005-11-01,2006-04-22
2,2006,2006-10-31,2007-04-21
3,2007,2007-10-30,2008-04-19
4,2008,2008-10-28,2009-04-18


## Test different models for all seasons

In [7]:
useful_columns = ["HOME_TEAM_ID", "VISITOR_TEAM_ID",
                    'FG_PCT_home', 'FT_PCT_home',
                    'FG3_PCT_home', 'AST_home', 'REB_home',
                    'FG_PCT_away', 'FT_PCT_away',
                    'FG3_PCT_away', 'AST_away', 'REB_away',
                    'HOME_TEAM_WINS']

In [4]:
model_list = [LogisticRegression, DecisionTreeClassifier, RandomForestClassifier,
              SVC, KNeighborsClassifier, GaussianNB]

tree_params = {'criterion': ['gini', 'entropy'],
                'max_depth': [i for i in range(2, 14, 2)]}

svc_params = {'kernel': ['linear', 'poly', 'rbf'],
              'degree': [i for i in range(3, 6)]}

knn_params = {'n_neighbors': [i for i in range(1, 6)],
              'weights': ['uniform', 'distance']}

train_results = {m: [] for m in model_list}
test_results = {m: [] for m in model_list}
for row in tqdm(season_schedule.iterrows()):
    season, _, __ = row[1].values
    X = df_games.loc[df_games["SEASON"] == season, useful_columns]
    cols_to_norm = ['AST_home', 'REB_home',
                    'AST_away', 'REB_away']
    X.loc[:, cols_to_norm] = min_max_scaler.fit_transform(X.loc[:, cols_to_norm])
    X_train, X_test, y_train, y_test = train_test_split(
                                                        X.drop(["HOME_TEAM_WINS"], axis=1),
                                                        X["HOME_TEAM_WINS"],
                                                        test_size=0.3, random_state=42)

    for model in model_list:
        current_model = model()
        if isinstance(current_model, (DecisionTreeClassifier, RandomForestClassifier)):
            current_model = GridSearchCV(estimator=current_model,
                                         cv=3, param_grid=tree_params,
                                         scoring='accuracy')
            current_model.fit(X_train, y_train)
            scores['test_accuracy'] = current_model.cv_results_['mean_test_score']
        elif isinstance(current_model, SVC):
            current_model = GridSearchCV(estimator=current_model,
                                         cv=3, param_grid=svc_params,
                                         scoring='accuracy')
            current_model.fit(X_train, y_train)
            scores['test_accuracy'] = current_model.cv_results_['mean_test_score']
        elif isinstance(current_model, KNeighborsClassifier):
            current_model = GridSearchCV(estimator=current_model,
                                         cv=3, param_grid=knn_params,
                                         scoring='accuracy')
            current_model.fit(X_train, y_train)
            scores['test_accuracy'] = current_model.cv_results_['mean_test_score']
        else:
            scores = cross_validate(current_model, X_train, y_train, cv=3, scoring=['accuracy'])
        current_model.fit(X_train, y_train)
        y_pred = current_model.predict(X_test)
        test_results[model].append(scores['test_accuracy'].mean())


18it [10:08, 33.78s/it]


In [5]:
mean_test_results = {m: np.array(res).mean() for m, res in test_results.items()}
mean_test_results = dict(sorted(mean_test_results.items(), key=lambda item: item[1], reverse=True))
mean_test_results

{sklearn.naive_bayes.GaussianNB: 0.8344038261845459,
 sklearn.ensemble._forest.RandomForestClassifier: 0.8172781734036932,
 sklearn.svm._classes.SVC: 0.8120149283917498,
 sklearn.linear_model._logistic.LogisticRegression: 0.8089121149240126,
 sklearn.tree._classes.DecisionTreeClassifier: 0.7510254795950048,
 sklearn.neighbors._classification.KNeighborsClassifier: 0.7129179584319467}

### Looks like Naive Bayes shows best results (by now), so let's test it 

## Compute average data for each team for a gven season

In [4]:
def compute_season_data(season_year:int=2018) -> pd.DataFrame:
    if 2004 > season_year > 2022:
        print("NO DATA FOR THIS SEASON")
        return
    
    season_start = season_schedule.loc[season_schedule["SEASON"] == season_year]["START"].iloc[0]
    season_end = season_schedule.loc[season_schedule["SEASON"] == season_year]["END"].iloc[0]
    season_data = df_games[(df_games["SEASON"] == season_year)
                           & (df_games["GAME_DATE_EST"] < season_end)].sort_values(by="GAME_DATE_EST")
    
    teams_data = df_teams[["TEAM_ID", "ABBREVIATION"]]
    teams_data.loc[:, ["SEASON"]] = season_year
    
    home_features = ['FG_PCT_home', 'FT_PCT_home',
                     'FG3_PCT_home', 'AST_home', 'REB_home']
    away_features = ['FG_PCT_away', 'FT_PCT_away',
                     'FG3_PCT_away', 'AST_away', 'REB_away']

    for team_row in teams_data.iterrows():
        idx, (team_id, _, __) = team_row[0], team_row[1].values
        home_data = season_data.loc[season_data["HOME_TEAM_ID"] == team_id, home_features]
        home_data = home_data.rename(columns={f: f[:-5] for f in home_features})

        away_data = season_data.loc[season_data["VISITOR_TEAM_ID"] == team_id, away_features]
        away_data = away_data.rename(columns={f: f[:-5] for f in away_features})
        
        temp_data = pd.concat([home_data, away_data])
        for col in temp_data.columns:
          teams_data.loc[idx, [col]] = temp_data.loc[:, [col]].mean()  
          
    cols_to_norm = ["AST", "REB"]
    teams_data.loc[:, cols_to_norm] = min_max_scaler.fit_transform(teams_data.loc[:, cols_to_norm])
    return teams_data


## Simulate one play-off series for 2 given teams

In [12]:
home_visitor_order = [True, True, False, False, True, False, True]
scale = 0.2

def simulate_play_off_round(team1: str, team2: str, ml_model, season_data: pd.DataFrame):
    if team1 == team2:
        print("YOU ENTERED THE SAME TEAMS")
        return
    
    team1_data = season_data.loc[season_data["ABBREVIATION"] == team1, :]
    team2_data = season_data.loc[season_data["ABBREVIATION"] == team2, :]
    
    home_team_id = []
    away_team_id = []

    fg_pct_home, ft_pct_home, fg3_pct_home = [], [], []
    ast_home, reb_home = [], []

    fg_pct_away, ft_pct_away, fg3_pct_away = [], [], []
    ast_away, reb_away = [], []
    for hv in home_visitor_order:
        if hv:
            home_team = team1_data
            away_team = team2_data
        else:
            home_team = team2_data
            away_team = team1_data
        
        home_team_id.append(home_team["TEAM_ID"].iloc[0])
        fg_pct_home.append(np.random.normal(home_team["FG_PCT"].iloc[0], scale=scale))
        ft_pct_home.append(np.random.normal(home_team["FT_PCT"].iloc[0], scale=scale))
        fg3_pct_home.append(np.random.normal(home_team["FG3_PCT"].iloc[0], scale=scale))
        ast_home.append(np.random.normal(home_team["AST"].iloc[0], scale=scale))
        reb_home.append(np.random.normal(home_team["REB"].iloc[0], scale=scale))
        
        away_team_id.append(away_team["TEAM_ID"].iloc[0])
        fg_pct_away.append(np.random.normal(away_team["FG_PCT"].iloc[0], scale=scale))
        ft_pct_away.append(np.random.normal(away_team["FT_PCT"].iloc[0], scale=scale))
        fg3_pct_away.append(np.random.normal(away_team["FG3_PCT"].iloc[0], scale=scale))
        ast_away.append(np.random.normal(away_team["AST"].iloc[0], scale=scale))
        reb_away.append(np.random.normal(away_team["REB"].iloc[0], scale=scale))
    
    cols = useful_columns
    data = {cols[0]: home_team_id, cols[1]: away_team_id,
            cols[2]: fg_pct_home, cols[3]: ft_pct_home, cols[4]: fg3_pct_home,
            cols[5]: ast_home, cols[6]: reb_home,
            cols[7]: fg_pct_away, cols[8]: ft_pct_away, cols[9]: fg3_pct_away,
            cols[10]: ast_away, cols[11]: reb_away}
    games = pd.DataFrame(data)
    results = ml_model.predict(games) 

    team1_wins, team2_wins = 0, 0
    for r, hv in zip(results, home_visitor_order):
        if r == 0:
            if hv:
                team2_wins += 1
            else:
                team1_wins += 1
        else:
            if hv:
                team1_wins += 1
            else:
                team2_wins += 1
        if team1_wins == 4:
            return team1
        elif team2_wins == 4:
            return team2

    return results

## Compare results between 2 teams in different seasons

In [19]:
def compare_results(team_pair: list, season_list: list) -> None:
    for season_to_simulate in season_list:
        print(f"Season {season_to_simulate}-{season_to_simulate+1}")
        start = season_schedule.loc[season_schedule["SEASON"] == season_to_simulate, ["START", "END"]]["START"].iloc[0]
        end = season_schedule.loc[season_schedule["SEASON"] == season_to_simulate, ["START", "END"]]["END"].iloc[0]
        X = df_games.loc[(end > df_games["GAME_DATE_EST"]) &
                         (df_games["GAME_DATE_EST"] >= start), useful_columns]
        cols_to_norm = ['AST_home', 'REB_home',
                        'AST_away', 'REB_away']
        X.loc[:, cols_to_norm] = min_max_scaler.fit_transform(X.loc[:, cols_to_norm])
        X, y = X.drop(["HOME_TEAM_WINS"], axis=1), X["HOME_TEAM_WINS"]

        best_model = GaussianNB()
        best_model.fit(X, y)
        season_data = compute_season_data(season_to_simulate)

        teams_results = {team: 0 for team in team_pair}
        num_of_sim = 100
        for i in range(num_of_sim):
            winner = simulate_play_off_round(team_pair[0], team_pair[1], best_model, season_data)
            teams_results[winner] += 1

        for t,w in teams_results.items():
            print(f"{t} won {w/num_of_sim*100:.3f}% of mathces")
        print()

year_list = [2015, 2019]
team_pair = ["LAL", "ATL"]
compare_results(team_pair, year_list)

Season 2015-2016
LAL won 17.000% of mathces
ATL won 83.000% of mathces

Season 2019-2020
LAL won 82.000% of mathces
ATL won 18.000% of mathces



### We can see that in different seasons different teams were stronger. Let's compare winning percantage from real data

In [22]:
df_standing = pd.read_csv("data/ranking.csv")
df_standing = df_standing.loc[:, ["TEAM_ID", "STANDINGSDATE", "TEAM", "W_PCT"]]
df_standing.loc[:, ["TEAM_ID"]] = min_max_scaler.fit_transform(df_standing.loc[:, ["TEAM_ID"]])
df_standing = pd.merge(df_standing, df_teams.loc[:, ["TEAM_ID", "ABBREVIATION"]], on="TEAM_ID")

print("Winning percentage")
for year in year_list:
    print(f"Season {year}-{year+1}")
    start = season_schedule.loc[season_schedule["SEASON"] == year]["START"].iloc[0]
    end = season_schedule.loc[season_schedule["SEASON"] == year]["END"].iloc[0]
    cur_season_data = df_standing.loc[df_standing["STANDINGSDATE"] == end].sort_values(by="TEAM")

    for t in team_pair:
        w_pct = cur_season_data.loc[cur_season_data["ABBREVIATION"] == t]["W_PCT"].iloc[0]
        print(f"{t}: {w_pct*100}%")
    print()

Winning percentage
Season 2015-2016
LAL: 20.7%
ATL: 58.5%

Season 2019-2020
LAL: 74.3%
ATL: 29.9%



### We can see that in season 2015-2016 `LAL` won only 20% of their mathces while `ATL` won 58% percent of their matches, so probably `ATL` was stronger in that season, this proves our prediction above. However, in the season 2019-2020 winning percantage is other way around, which means `LAL` was stronger that season, as well as according to our prediction.